# Part 1: Voting behavior
Questions:
- What is the average participation rate?
- Which proposals gather the most participation? (both in terms of individual votes and total veCRV)
- Are proposals mostly one sided?
- What are the most controversial proposals?

In [1]:
import json
from subgrounds import Subgrounds, SyntheticField

sg = Subgrounds()
curve_voting = sg.load_subgraph("https://api.thegraph.com/subgraphs/name/convex-community/curve-dao")

In [2]:
# Voting ratio and controversy index
curve_voting.Proposal.vote_ratio = curve_voting.Proposal.votesFor / (curve_voting.Proposal.votesAgainst + curve_voting.Proposal.votesFor)
curve_voting.Proposal.controversy = (0.5 - abs(curve_voting.Proposal.vote_ratio - 0.5))/0.5
curve_voting.Proposal.req_support_percent = curve_voting.Proposal.supportRequired / 10**18

# Voting participation
curve_voting.Proposal.participation = (curve_voting.Proposal.votesAgainst + curve_voting.Proposal.votesFor) / curve_voting.Proposal.totalSupply
curve_voting.Proposal.req_quorum_percent = curve_voting.Proposal.minAcceptQuorum / 10**18

curve_voting.Proposal.datetime = SyntheticField.datetime_of_timestamp(curve_voting.Proposal.startDate)

def description_of_metadata(metadata: str) -> str:
    meta = json.loads(metadata)
    return meta["text"]


proposals_fp = curve_voting.Query.proposals(
    first=5000,
    orderBy=curve_voting.Proposal.startDate,
    orderDirection="desc",
)

proposals = sg.query_df([
    proposals_fp.id,
    proposals_fp.datetime,
    proposals_fp.vote_ratio,
    proposals_fp.req_support_percent,
    proposals_fp.controversy,

    proposals_fp.voteCount,
    proposals_fp.participation,
    proposals_fp.req_quorum_percent,

    proposals_fp.executed,
    proposals_fp.metadata,
])

### Summary statistics

In [3]:
proposals[["proposals_vote_ratio", "proposals_controversy", "proposals_voteCount", "proposals_participation"]].describe()

,proposals_vote_ratio,proposals_controversy,proposals_voteCount,proposals_participation
count,393.000000,393.000000,393.000000,393.000000
mean,0.918986,0.038246,20.865140,0.453434
std,0.245648,0.134277,45.668388,0.214837
min,0.000000,0.000000,0.000000,0.000000
25%,0.998445,0.000000,5.000000,0.329174
50%,1.000000,0.000000,12.000000,0.500435
75%,1.000000,0.001632,28.000000,0.614105
max,1.000000,0.988906,822.000000,0.833622


### Highest Participation (veCRV amount)

In [4]:
most_participated_proposals = proposals.sort_values(["proposals_participation"], ascending=False).reset_index(drop=True).head(10)
most_participated_proposals

,proposals_id,proposals_datetime,proposals_vote_ratio,proposals_req_support_percent,proposals_controversy,proposals_voteCount,proposals_participation,proposals_req_quorum_percent,proposals_executed,proposals_metadata
0,0xe478de485ad2fe566d49342cbd03e49ed7db3356-283,2023-02-13 11:15:59,1.000000,0.51,0.000000,66,0.833622,0.30,True,"""Add EMA-enabled 2 coin pool implementations (..."
1,0xe478de485ad2fe566d49342cbd03e49ed7db3356-296,2023-03-08 08:32:11,1.000000,0.51,0.000000,30,0.828097,0.30,True,Add a gauge for the following pool: cvxCrv wit...
2,0xe478de485ad2fe566d49342cbd03e49ed7db3356-273,2023-02-07 18:03:59,0.959484,0.51,0.081031,33,0.821586,0.30,True,Add a gauge for the following pool: OGV/ETH [R...
3,0xe478de485ad2fe566d49342cbd03e49ed7db3356-295,2023-03-08 05:40:47,1.000000,0.51,0.000000,30,0.811040,0.30,False,"""Increase EMA time for new CRV/yCRV pool"
4,0xe478de485ad2fe566d49342cbd03e49ed7db3356-260,2023-01-18 15:18:23,1.000000,0.51,0.000000,36,0.808192,0.30,True,Add a gauge for the following pool: MIM+3crv o...
5,0xbcff8b0b9419b9a88c44546519b1e909cf330399-60,2023-02-07 16:15:47,1.000000,0.60,0.000000,8,0.794121,0.15,True,"""Apply new parameters for MATIC/ETH changes"
6,0xe478de485ad2fe566d49342cbd03e49ed7db3356-240,2022-12-13 19:38:23,0.999219,0.51,0.001562,39,0.776004,0.30,True,"""Add wbtc/sbtc v2 gauge to the gauge controlle..."
7,0xe478de485ad2fe566d49342cbd03e49ed7db3356-231,2022-11-18 07:07:23,1.000000,0.51,0.000000,27,0.770028,0.30,True,Add a gauge for the following pool: SDT / FraxBP
8,0xe478de485ad2fe566d49342cbd03e49ed7db3356-297,2023-03-10 04:52:11,1.000000,0.51,0.000000,29,0.760830,0.30,True,"""Add new CRV/yCRV gauge to gauge controller."
9,0xe478de485ad2fe566d49342cbd03e49ed7db3356-213,2022-09-19 10:41:35,1.000000,0.51,0.000000,35,0.744985,0.30,True,Add a gauge for the following pool: cbETH/ETH ...


### Highest Participation (individual votes)

In [5]:
most_voted_proposals = proposals.sort_values(["proposals_voteCount"], ascending=False).reset_index(drop=True).head(10)
most_voted_proposals

,proposals_id,proposals_datetime,proposals_vote_ratio,proposals_req_support_percent,proposals_controversy,proposals_voteCount,proposals_participation,proposals_req_quorum_percent,proposals_executed,proposals_metadata
0,0xe478de485ad2fe566d49342cbd03e49ed7db3356-44,2021-04-23 09:25:23,1.000000,0.51,0.000000e+00,822,0.599126,0.3,True,"""Whitelist the Convex protocol"
1,0xe478de485ad2fe566d49342cbd03e49ed7db3356-35,2021-01-31 16:00:15,1.000000,0.51,0.000000e+00,133,0.443939,0.3,True,whitelist StakeDAO CurveYCrvVoter in Curveâs...
2,0xe478de485ad2fe566d49342cbd03e49ed7db3356-188,2022-06-15 12:22:54,1.000000,0.51,0.000000e+00,132,0.474638,0.3,True,"""Add burner for new Factory Crypto Pools."
3,0xe478de485ad2fe566d49342cbd03e49ed7db3356-45,2021-04-27 09:53:14,1.000000,0.51,0.000000e+00,124,0.402830,0.3,True,"""Enable claiming and distribution of AAVE to l..."
4,0xe478de485ad2fe566d49342cbd03e49ed7db3356-48,2021-06-02 19:18:57,0.999803,0.51,3.947108e-04,120,0.387932,0.3,True,"""Add a new gauge type for Polygon, and a gauge..."
5,0xe478de485ad2fe566d49342cbd03e49ed7db3356-187,2022-06-13 08:58:11,0.999203,0.51,1.593474e-03,107,0.443956,0.3,True,Add a gauge for the following pool: FRAX/USDC
6,0xe478de485ad2fe566d49342cbd03e49ed7db3356-81,2021-09-14 22:21:36,0.095298,0.51,1.905962e-01,105,0.393050,0.3,False,"""Whitelist Earning.Farm protocol in Curve\u201..."
7,0xe478de485ad2fe566d49342cbd03e49ed7db3356-47,2021-06-02 19:03:27,0.999797,0.51,4.059265e-04,105,0.389996,0.3,True,"""Add a new gauge type for Fantom, and gauges f..."
8,0xe478de485ad2fe566d49342cbd03e49ed7db3356-29,2020-12-01 15:33:58,0.999975,0.51,4.987283e-05,102,0.358731,0.3,True,Enable token checkpoint in the FeeDistributor ...
9,0xe478de485ad2fe566d49342cbd03e49ed7db3356-28,2020-11-20 08:36:27,1.000000,0.51,1.596903e-08,95,0.497453,0.3,True,Set converters for admin fees to start distrib...


### Most Controversial Proposals

In [6]:
controversial_proposals = proposals.sort_values(["proposals_controversy"], ascending=False).reset_index(drop=True).head(10)
controversial_proposals

,proposals_id,proposals_datetime,proposals_vote_ratio,proposals_req_support_percent,proposals_controversy,proposals_voteCount,proposals_participation,proposals_req_quorum_percent,proposals_executed,proposals_metadata
0,0xe478de485ad2fe566d49342cbd03e49ed7db3356-114,2021-12-31 19:10:18,0.505547,0.51,0.988906,41,0.656479,0.30,False,Add a gauge for the following pool: FRAX/UST (...
1,0xe478de485ad2fe566d49342cbd03e49ed7db3356-19,2020-10-06 05:43:19,0.437204,0.51,0.874408,15,0.045582,0.30,False,Add a flat 0.040% withdrawal fee to all pools....
2,0xbcff8b0b9419b9a88c44546519b1e909cf330399-6,2020-09-28 09:11:42,0.427574,0.60,0.855149,31,0.121555,0.15,False,Increase the current 0.04% trading fee to .05%...
3,0xe478de485ad2fe566d49342cbd03e49ed7db3356-182,2022-05-04 09:35:49,0.581338,0.51,0.837323,20,0.708433,0.30,True,Add a gauge for the following pool: USDD/3CRV ...
4,0xe478de485ad2fe566d49342cbd03e49ed7db3356-218,2022-09-26 08:17:11,0.588037,0.51,0.823926,20,0.729214,0.30,True,Kill CRV rewards for the CNC/ETH gauge since t...
5,0xe478de485ad2fe566d49342cbd03e49ed7db3356-216,2022-09-23 11:24:23,0.338622,0.51,0.677244,18,0.618504,0.30,False,Add a gauge for the following pool: CRV/yCRV
6,0xbcff8b0b9419b9a88c44546519b1e909cf330399-38,2022-06-01 07:11:50,0.666349,0.60,0.667302,19,0.577107,0.15,True,"""Ramp A from 50 to 25 for PUSD metapool over 3..."
7,0xe478de485ad2fe566d49342cbd03e49ed7db3356-83,2021-09-15 10:47:33,0.327515,0.51,0.655030,20,0.543017,0.30,False,Add a gauge for the following pool: sdveCRV+CRV
8,0xe478de485ad2fe566d49342cbd03e49ed7db3356-13,2020-09-30 00:23:48,0.292756,0.51,0.585513,17,0.109380,0.30,False,Add .005% trading fee which will market buy an...
9,0xbcff8b0b9419b9a88c44546519b1e909cf330399-42,2022-06-28 03:56:19,0.723920,0.60,0.552161,15,0.704244,0.15,True,"""Ramp A from 10 to 25 for sdCRV/CRV pool over ..."


# Part 2: Voting and Protocol Health
Questions:
- How does the outcome of proposal voting affect the protocol? (w.r.t.: TVL, volume, fees)

Note: We'll focus on Ethereum, but good exercise would be to generalize to multiple chains using Messari subgraphs

In [7]:
curve_ethereum = sg.load_subgraph("https://api.thegraph.com/subgraphs/name/messari/curve-finance-ethereum")

In [8]:
from subgrounds.contrib.plotly import Figure, Scatter

curve_voting.Proposal.participation = (curve_voting.Proposal.votesAgainst + curve_voting.Proposal.votesFor) / curve_voting.Proposal.totalSupply

proposals_fp = curve_voting.Query.proposals(
    first=5000,
    orderBy=curve_voting.Proposal.startDate,
    orderDirection="desc",
    where=[
        curve_voting.Proposal.executed == True
    ]
)

curve_ethereum.FinancialsDailySnapshot.datetime = SyntheticField.datetime_of_timestamp(curve_ethereum.FinancialsDailySnapshot.timestamp)

financial_metrics_fp = curve_ethereum.Query.financialsDailySnapshots(
    first=5000,
    orderBy=curve_ethereum.FinancialsDailySnapshot.timestamp,
    orderDirection="desc",
)

proposals_tvl = Figure(
    subgrounds=sg,
    traces=[
        Scatter(
            x=proposals_fp.datetime,
            y=proposals_fp.participation,
            text=proposals_fp.metadata,
            name="Proposals",
            mode="markers",
        ),
        Scatter(
            x=financial_metrics_fp.datetime,
            y=financial_metrics_fp.totalValueLockedUSD,
            name="Total Value Locked",
            yaxis="y2",
        ),
    ],
    layout={
        "showlegend": True,
        "hovermode": "x",
        "spikedistance": -1,
        "xaxis": {
            "showspikes": True,
            "spikedash": "solid",
            "spikemode": "across+toaxis",
            "spikesnap": "cursor",
            "spikecolor": "black",
            "spikethickness": 1,
            "showline": True,
            "showgrid": True,
        },
        "yaxis": {
            "title": "Proposals",
            "titlefont": {"color": "#1f77b4"},
            "tickfont": {"color": "#ffffff"},
        },
        "yaxis2": {
            "title": "Total Value Locked",
            "titlefont": {"color": "#ff7f0e"},
            "tickfont": {"color": "#ff7f0e"},
            "anchor": "free",
            "overlaying": "y",
            "side": "left",
            "position": 0.05,
        },
    }
)

proposals_tvl.figure

# Part 3: User DAO activity history
Questions:
- Given a user, what did they vote on in the past? Did they create proposals? When did they lock CRV to be able to vote?
- Are there proposal votes that prompted a user to lock more veCRV?

Good users to showcase viz:
- 0x4810939a9f66e80ef5f98e4f95a70710d121ad61
- 0x7a16ff8270133f063aab6c9977183d9e72835428

In [9]:
import plotly.graph_objects as go


curve_voting.Proposal.datetime = SyntheticField.datetime_of_timestamp(curve_voting.Proposal.startDate)
curve_voting.Vote.datetime = SyntheticField.datetime_of_timestamp(curve_voting.Vote.timestamp)
curve_voting.GaugeWeightVote.datetime = SyntheticField.datetime_of_timestamp(curve_voting.GaugeWeightVote.timestamp)
curve_voting.Lock.datetime = SyntheticField.datetime_of_timestamp(curve_voting.Lock.timestamp)

curve_voting.Proposal.type = SyntheticField.constant("PROPOSAL")
curve_voting.Vote.type = SyntheticField.constant("VOTE")
curve_voting.GaugeWeightVote.type = SyntheticField.constant("GAUGE_VOTE")
curve_voting.Lock.type = SyntheticField.constant("LOCK")

curve_voting.Lock.adj_amount = curve_voting.Lock.totalLocked / 10**18

def voter_history_viz(voter_address: str) -> go.Figure:
    voter_fp = curve_voting.Query.user(
        id=voter_address
    )

    voter_proposals_fp = voter_fp.proposals(first=1000)
    voter_votes_fp = voter_fp.votes(first=1000)
    voter_gauge_votes_fp = voter_fp.gaugeWeightVotes(first=1000)
    voter_locks_fp = voter_fp.locks(first=1000, orderBy=curve_voting.Lock.timestamp, orderDirection="asc")

    voter_proposals = sg.query_df([
        voter_proposals_fp.datetime,
        voter_proposals_fp.metadata,
        voter_proposals_fp.type,
    ])

    voter_votes = sg.query_df([
        voter_votes_fp.datetime,
        voter_votes_fp.proposal.metadata,
        voter_votes_fp.type,
    ])

    voter_gauge_weight_votes = sg.query_df([
        voter_gauge_votes_fp.datetime,
        voter_gauge_votes_fp.weight,
        voter_gauge_votes_fp.gauge,
        voter_gauge_votes_fp.type,
    ])

    voter_locks = sg.query_df([
        voter_locks_fp.datetime,
        voter_locks_fp.value,
        voter_locks_fp.type,
        voter_locks_fp.adj_amount
    ])

    return go.Figure([
        go.Scatter(
            name="Proposals",
            x=voter_proposals.user_proposals_datetime,
            y=voter_proposals.user_proposals_type, 
            text=voter_proposals.user_proposals_metadata,
            mode="markers"
        ) if len(voter_proposals) > 0 else go.Scatter(name="Proposals", x=[], y=[]),
        
        go.Scatter(
            name="Proposal votes",
            x=voter_votes.user_votes_datetime,
            y=voter_votes.user_votes_type, 
            text=voter_votes.user_votes_proposal_metadata,
            mode="markers"
        ) if len(voter_votes) > 0 else go.Scatter(x=[], y=[]),

        go.Scatter(
            name="Gauge weight votes",
            x=voter_gauge_weight_votes.user_gaugeWeightVotes_datetime,
            y=voter_gauge_weight_votes.user_gaugeWeightVotes_type, 
            text=voter_gauge_weight_votes.user_gaugeWeightVotes_gauge,
            mode="markers"
        ) if len(voter_gauge_weight_votes) > 0 else go.Scatter(name="Gauge weight votes", x=[], y=[]),

        go.Scatter(
            name="veCRV locks",
            x=voter_locks.user_locks_datetime,
            y=voter_locks.user_locks_type, 
            text=voter_locks.user_locks_value,
            mode="markers"
        ) if len(voter_locks) > 0 else go.Scatter(name="veCRV locks", x=[], y=[]),

        go.Scatter(
            name="veCRV balance",
            x=voter_locks.user_locks_datetime,
            y=voter_locks.user_locks_adj_amount,
            yaxis="y2",
            fill='tozeroy',
        ) if len(voter_locks) > 0 else go.Scatter(x=[], y=[]),
    ],
    layout={
        "showlegend": True,
        "hovermode": "x",
        "spikedistance": -1,
        "xaxis": {
            "showspikes": True,
            "spikedash": "solid",
            "spikemode": "across+toaxis",
            "spikesnap": "cursor",
            "spikecolor": "black",
            "spikethickness": 1,
            "showline": True,
            "showgrid": True,
        },
        "yaxis": {
            "title": "DAO Activity",
            "titlefont": {"color": "#1f77b4"},
            "tickfont": {"color": "#ffffff"},
        },
        "yaxis2": {
            "title": "veCRV balance",
            "titlefont": {"color": "#ff7f0e"},
            "tickfont": {"color": "#ff7f0e"},
            "anchor": "free",
            "overlaying": "y",
            "side": "left",
            "position": 0.05,
        },
    },
)

voter_history_viz("0x7a16ff8270133f063aab6c9977183d9e72835428")

# Part 4: Multi-chart
Questions:
- Given a proposal, how is voting distributed?

Note: The goal here is to showcase subplots, the analysis itself is not the most interesting

In [10]:
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import pandas as pd


# Synthetic fields
curve_voting.Vote.adj_stake = curve_voting.Vote.stake / 10**18

def proposal_voters_viz(proposal_id: str) -> go.Figure:
    # Query data
    votes_fp = curve_voting.Query.votes(
        first=1000,
        orderBy=curve_voting.Vote.stake,
        orderDirection="desc",
        where=[
            curve_voting.Vote.proposal == proposal_id
        ]
    )

    votes: pd.DataFrame = sg.query_df([
        votes_fp.voter.id,
        votes_fp.adj_stake,
        votes_fp.supports,
    ])

    # Create viz
    fig = make_subplots(
        rows=2, cols=3, 
        specs=[
            [{"rowspan": 2, "colspan": 2, "type": "table"}, None, {"type": "pie"}],
            [None, None, {"type": "histogram"}]
        ],
    )

    fig.add_trace(
        go.Table(
            header={"values": ["Voter address", "Stake", "Support"]},
            cells={"values": [votes[k].tolist() for k in votes.columns]},
            columnwidth = [64, 50, 50],
        ),
        row=1, col=1
    )

    pie_data = pd.concat([
        votes.sort_values(["votes_adj_stake"], ascending=False).iloc[:10],
        pd.DataFrame([{
            "votes_voter_id": "Others",
            "votes_adj_stake": votes.sort_values(["votes_adj_stake"], ascending=False).iloc[10:].sum(),
            "votes_supports": True
        }])
    ])

    fig.add_trace(
        go.Pie(
            labels=pie_data["votes_voter_id"],
            values=pie_data["votes_adj_stake"]
        ),
        row=1, col=3
    )

    fig.add_trace(
        go.Histogram(
            name="Stake Size",
            x=votes["votes_adj_stake"],
            xbins={"start": 0.0, "size": 100_000},
            
        ),
        row=2, col=3
    )
    fig.update_layout(showlegend=False)
    return fig

proposal_voters_viz("0xe478de485ad2fe566d49342cbd03e49ed7db3356-44").show()